# Roche robotics poject

## Imports

In [1]:
import random

import numpy as np
import matplotlib.pyplot as plt
from omegaconf import OmegaConf
import torch
import torch.nn.functional as F
import json
import pandas as pd
import os

from habitat_baselines.config.default import get_config
from habitat_baselines.rl.ppo.ppo_trainer import PPOTrainer
from habitat_baselines.common.construct_vector_env import construct_envs
from habitat_baselines.config.default import get_config

from pg.base_pg import BasePolicyGradient
from pg.base_pg_trainer import BasePolicyGradientTrainer

from tensorboard.backend.event_processing import event_accumulator

Neither `ifconfig` (`ifconfig -a`) nor `ip` (`ip address show`) commands are available, listing network interfaces is likely to fail


## Useful functions

In [2]:
def parse_tensorboard(path, scalars, prefix=""):
    """returns a dictionary of pandas dataframes for each requested scalar"""
    ea = event_accumulator.EventAccumulator(
        path,
        size_guidance={event_accumulator.SCALARS: 0},
    )
    _ = ea.Reload()
    df = pd.DataFrame(ea.Scalars(scalars[0]))
    df[f'{scalars[0]}'] = df['value']
    df.drop(columns=['value'], inplace=True)
    for k in scalars:
        df[k] = pd.DataFrame(ea.Scalars(k))['value']
    return df

## Configuration 1 : PointNav, Empty Room

### PPO Configuration

In [5]:
def build_PPO_config():    
    config = get_config("pointnav/ppo_pointnav.yaml")
    OmegaConf.set_readonly(config, False)
    config.habitat_baselines.tensorboard_dir = "tb/PPO"
    config.habitat_baselines.checkpoint_folder = "data/PPO2_checkpoints"
    config.habitat_baselines.num_updates = -1
    config.habitat_baselines.num_environments = 2
    config.habitat_baselines.verbose = True
    config.habitat_baselines.num_checkpoints = -1
    config.habitat_baselines.checkpoint_interval = 1000000
    config.habitat_baselines.total_num_steps = 150 * 1000
    config.habitat.dataset.data_path = "data/datasets/pointnav/simple_room/v0/{split}/empty_room.json.gz"
    config.habitat_baselines.load_resume_state_config = False
    #config.habitat.simulator.scene = "data/scene_datasets/simple_room/empty_room.glb"

    OmegaConf.set_readonly(config, True)

    return config

config = build_PPO_config()
om_config = OmegaConf.to_yaml(config)
#print(om_config)
      
# Set randomness
random.seed(config.habitat.seed)
np.random.seed(config.habitat.seed)
torch.manual_seed(config.habitat.seed)
if (
    config.habitat_baselines.force_torch_single_threaded
    and torch.cuda.is_available()
):
    torch.set_num_threads(1)

os.environ["MAGNUM_LOG"] = "quiet"
os.environ["HABITAT_SIM_LOG"] = "quiet"

### Training phase

In [6]:
# Build the trainer and start training
trainer = PPOTrainer(config)
trainer.train()

2025-05-02 09:45:06,434 Loading resume state: data/PPO2_checkpoints/.habitat-resume-state.pth
2025-05-02 09:45:06,521 config: habitat:
  seed: 100
  env_task: GymHabitatEnv
  env_task_gym_dependencies: []
  env_task_gym_id: ''
  environment:
    max_episode_steps: 500
    max_episode_seconds: 10000000
    iterator_options:
      cycle: true
      shuffle: true
      group_by_scene: true
      num_episode_sample: -1
      max_scene_repeat_episodes: -1
      max_scene_repeat_steps: 10000
      step_repetition_range: 0.2
  simulator:
    type: Sim-v0
    action_space_config: v0
    action_space_config_arguments: {}
    forward_step_size: 0.25
    create_renderer: false
    requires_textures: true
    auto_sleep: false
    step_physics: true
    concur_render: false
    needs_markers: true
    update_robot: true
    scene: data/scene_datasets/habitat-test-scenes/van-gogh-room.glb
    scene_dataset: default
    additional_object_paths: []
    seed: ${habitat.seed}
    turn_angle: 10
    til

FileNotFoundError: [Errno 2] No such file or directory: 'checkpoint'

In [ ]:
ppo_config_1_logs = parse_tensorboard("tb/PPO", ["metrics/spl", "reward", "learner/grad_norm"])

### Agent sensors

In [ ]:
config = build_PPO_config()
OmegaConf.set_readonly(config, False)

# Creating the environment
envs = construct_envs(config)

# Retrieve the observations
obs = envs.reset()  

# Verify the keys of the observation
print("Available observations :", obs[0].keys())  # Obs of environment 0
OmegaConf.set_readonly(config, True)


## Configuration 2 : PointNav, Multiple Rooms

### PPO Configuration

In [ ]:
def build_PPO_config():
    config = get_config("pointnav/ppo_pointnav.yaml")
    OmegaConf.set_readonly(config, False)
    config.habitat_baselines.tensorboard_dir = "tb/PPO"
    config.habitat_baselines.num_updates = -1
    config.habitat_baselines.num_environments = 2
    config.habitat_baselines.verbose = True
    config.habitat_baselines.num_checkpoints = -1
    config.habitat_baselines.checkpoint_interval = 1000000
    config.habitat_baselines.total_num_steps = 150 * 1000
    config.habitat.dataset.data_path="data/datasets/pointnav/multiple_room/v0/{split}/Adrian.json.gz"
    OmegaConf.set_readonly(config, True)

    return config

config = build_PPO_config()  # Build the config for PPO
# Set randomness
random.seed(config.habitat.seed)
np.random.seed(config.habitat.seed)
torch.manual_seed(config.habitat.seed)
if (
    config.habitat_baselines.force_torch_single_threaded
    and torch.cuda.is_available()
):
    torch.set_num_threads(1)

os.environ["MAGNUM_LOG"] = "quiet"
os.environ["HABITAT_SIM_LOG"] = "quiet"

### Training phase

In [ ]:
# Build the trainer and start training
trainer = PPOTrainer(config)
trainer.train()

In [ ]:
ppo_config_2_logs = parse_tensorboard("tb/PPO", ["metrics/spl", "reward", "learner/grad_norm"])

## Perfomances

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(6, 6))
plt.subplots_adjust(hspace=0.5)
plt.axes(axes[0])

plt.plot(ppo_config_1_logs['metrics/spl'], label="PPO")
plt.plot(ppo_config_2_logs['metrics/spl'], label="PPO")

plt.title("SPL")
plt.xlabel("Steps")
plt.legend()

plt.axes(axes[1])

plt.plot(ppo_config_1_logs['learner/grad_norm'], label="PPO")
plt.plot(ppo_config_2_logs['learner/grad_norm'], label="PPO")

plt.title("Gradient Norm")
plt.xlabel("Steps")
plt.legend()